In [ ]:
!pip install wikipedia

In [ ]:
import networkx as nx
import wikipedia

def generate_wikipedia_network(seed_pages, depth=1):
    graph = nx.DiGraph()

    def get_page_links(page_title):
        try:
            print(f"Fetching links for: {page_title}")
            page = wikipedia.page(page_title)
            links = page.links
            return links
        except wikipedia.exceptions.DisambiguationError as e:
            return [e.options[0]]
        except wikipedia.exceptions.PageError:
            return []

    def add_nodes_and_edges(parent_node, depth):
        if depth > 0:
            links = get_page_links(parent_node)
            for link in links:
                print(f"Adding node: {link}")
                graph.add_node(link)
                graph.add_edge(parent_node, link)
                add_nodes_and_edges(link, depth - 1)

    for seed_page in seed_pages:
        print(f"Adding seed node: {seed_page}")
        graph.add_node(seed_page)
        add_nodes_and_edges(seed_page, depth)

    return graph

# Specify seed pages
seed_pages = ['Python (programming language)', 'Ruby (programming language)', 'Rust (programming language)', 'Go (programming language)']

# Set depth limit to 1 (layers 0 and 1)
depth_limit = 1

# Generate Wikipedia network
print("Generating Wikipedia Network...")
wikipedia_network = generate_wikipedia_network(seed_pages, depth_limit)

# Export the graph to GEXF format
gexf_filename = "/content/wikipedia_network.gexf"
nx.write_gexf(wikipedia_network, gexf_filename)


# Calculate k-core and k-shell
k_core_values = nx.core_number(wikipedia_network)

# Add k-core and k-shell attributes to nodes
nx.set_node_attributes(wikipedia_network, k_core_values, 'k_core')

# Manually calculate k-shell values
k_shell_values = dict()
for node in wikipedia_network.nodes():
    k_shell_values[node] = len(list(nx.ego_graph(wikipedia_network, node, undirected=True).nodes()))

# Add k-shell attributes to nodes
nx.set_node_attributes(wikipedia_network, k_shell_values, 'k_shell')

# Export the graph to GEXF format
nx.write_gexf(wikipedia_network, "wikipedia_network.gexf")

# Download the GEXF file to your local machine
from google.colab import files
files.download(gexf_filename)
